# Extract profiles for all proplyds and all filters

## Library imports

### General libraries

In [2]:
import numpy as np 
from pathlib import Path
import pandas as pd

### Astronomy libraries

In [17]:
from astropy.coordinates import SkyCoord, Angle
import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table, QTable
import regions

### Graphics libraries

In [1]:
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
sns.set_context("talk")
sns.set_color_codes()

## Paths to the data files

In [5]:
datapath = Path.cwd().parent / "data"

## Get all the images we need in various filters

In [31]:
pcfilters = ["f631n", "f673n", "f656n", "f658n", "f547m", ]

In [35]:
class FilterImage:
    """WFPC2 PC image in a given filter
    
    Includes the following attributes:
    
    name: name of filter
    data: pixel image (hi-pass filtered)
    sdata: pixel image (lo-pass filtered)
    coords: celestial coordinates of pixels (same shape as data)
    wcs: an astropy.wcs.WCS instance
    """
    def __init__(self, name):
        self.name = name
        hdu = fits.open(
            datapath / f"align-pcmos-{self.name}_sharp_16.fits"
        )[0]
        self.wcs = WCS(hdu.header)
        self.data = hdu.data
        self.sdata = fits.open(
            datapath / f"pcmos-{self.name}_smooth_16.fits"
        )[0].data
        ny, nx = self.data.shape
        self.coords = self.wcs.pixel_to_world(
            *np.meshgrid(np.arange(nx), np.arange(ny))
        )
        

In [32]:
imdict = {
    name: FilterImage(name) for name in pcfilters
}

## Get all the proplyd source coordinates

Get the sources from the DS9 region file that I made by hand.

In [6]:
regfile = datapath / "pcmos-proplyds.reg"
regs = regions.Regions.read(regfile, format="ds9")

Use θ¹ Ori C as origin to define position angles.

In [7]:
c0 = SkyCoord.from_name("* tet01 Ori C")

Extract the information that we want from the region file. Construct a list of dicts that give source name and coordinates:

In [8]:
source_list = [{"Name": r.meta["label"],  "ICRS": r.center} for r in regs]

Convert to an `astropy.table.QTable` of the sources. Add columns for PA to θ¹ Ori C (in degrees) and Separation from θ¹ Ori C (in arcsec):

In [18]:
source_table = QTable(source_list)
source_table["PA"] = source_table["ICRS"].position_angle(c0).to(u.deg)
source_table["Sep"] = source_table["ICRS"].separation(c0).to(u.arcsec)
source_table.add_index("Name")
source_table

Name,ICRS,PA,Sep
,"deg,deg",deg,arcsec
str11,SkyCoord,float64,float64
177-341W,"83.82363499999998,-5.394699444444445",314.98007715439655,25.432600010148143
173-341,"83.82216416666665,-5.394801388888889",325.26623943671575,22.321924813790307
170-337,"83.82073749999998,-5.393605833333334",331.55791584978465,15.967686371231176
171-340,"83.82105708333333,-5.394350833333334",332.3781956253825,18.873422933876565
180-331,"83.82514916666666,-5.391885555555556",288.5266971454808,24.69660035312236
176-325,"83.82313874999998,-5.390233888888889",276.6893443122159,16.322331463594164
168-326,"83.82016916666666,-5.390601388888889",300.0748250386534,6.4342047406945815
161-328,"83.81696666666664,-5.391016944444445",51.38552690888274,7.563771376918397


By turning the `Name` column into an index, we can extract a given source by name. For example:

In [19]:
source_table.loc["182-413"]

Name,ICRS,PA,Sep
,"deg,deg",deg,arcsec
str11,SkyCoord,float64,float64
182-413,"83.82584708333331,-5.403739722222222",332.8419926483634,56.78242731489862


The advantage of using a `QTable` is that the units remain attached to the values:

In [22]:
source_table.loc["177-341W"]["PA"], source_table.loc["180-331"]["Sep"]

(<Angle 314.98007715 deg>, <Angle 24.69660035 arcsec>)

## Class to represent combination of a proplyd and an image in a particular filter

In [41]:
class ProplydImage:
    
    def __init__(self, pdata, image: FilterImage):
        self.c = pdata["ICRS"]
        self.pa = pdata["PA"]
        self.sep = pdata["Sep"]
        self.image = image

    def set_mask(
        self,
        r_out = 1.0 * u.arcsec,
        r_in = 0.1 * u.arcsec,
        mu_min = -0.2,
    ):
        r = self.c.separation(self.image.coords)
        pa = self.c.position_angle(self.image.coords)
        cth = np.cos((pa - self.pa))
        self.mask = (r <= r_out) & ((cth >= mu_min) | (r <= r_in))

## Do the profiles

In [ ]:
source_data = source_table.loc["177-341W"]


## Do the images